In [1]:
from __future__ import print_function, division, absolute_import
import os
import sys
sys.path.insert(0, "C:\\Users\\Matt\\Desktop\\orbithunter\\")
from orbithunter import *
import numpy as np
import itertools
from math import pi
import matplotlib.pyplot as plt
from orbithunter.arrayops import swap_modes
from scipy.sparse.linalg import lsqr, lsmr, LinearOperator
from scipy.optimize import  root, minimize
import time
import pandas as pd

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
def _scipy_sparse_linalg_solver_wrapper(orbit_, damp=0.0, atol=1e-03, btol=1e-03,
                                        method='lsqr', maxiter=None, conlim=1e+08,
                                        show=False, calc_var=False, **kwargs):

    linear_operator_shape = (orbit_.state.size, orbit_.state_vector().size)
    istop = 1
    n_iter = 0
    orbit_n_iter = 0
    # Return codes that represent good results from the SciPy least-squares solvers.
    good_codes = [0, 1, 2, 4, 5]
    residual = orbit_.residual()

    orbit_tol = kwargs.get('orbit_tol', orbit_.M * orbit_.N * 10**-6)
    orbit_maxiter = kwargs.get('orbit_maxiter', 250)
    max_damp_factor = kwargs.get('orbit_damp_max', 8)
    preconditioning = kwargs.get('preconditioning', False)

    while (residual > orbit_tol) and (istop in good_codes) and orbit_n_iter < orbit_maxiter:
        orbit_n_iter += 1

        # The operator depends on the current state; A=A(orbit)
        def rmv_func(v):
            # _process_newton_step turns state vector into class object.
            v_orbit = orbit_.from_numpy_array(v, **kwargs)
            v_orbit.T, v_orbit.L, v_orbit.S = orbit_.T, orbit_.L, orbit_.S
            rmatvec_orbit = orbit_.rmatvec(v_orbit, **kwargs)
            if preconditioning:
                return rmatvec_orbit.precondition(orbit_.parameters).state_vector().reshape(-1, 1)
            else:
                return rmatvec_orbit.state_vector().reshape(-1, 1)

        def mv_func(v):
            # _state_vector_to_orbit turns state vector into class object.
            v_orbit = orbit_.from_numpy_array(v, **kwargs)
            v_orbit.T, v_orbit.L, v_orbit.S = orbit_.T, orbit_.L, orbit_.S
            matvec_orbit = orbit_.matvec(v_orbit, **kwargs)
            if preconditioning:
                return matvec_orbit.precondition(orbit_.parameters).state.reshape(-1, 1)
            else:
                return matvec_orbit.state.reshape(-1, 1)

        orbit_linear_operator = LinearOperator(linear_operator_shape, mv_func, rmatvec=rmv_func, dtype=float)
        b = -1.0 * orbit_.spatiotemporal_mapping(**kwargs).state.reshape(-1, 1)
        damp_factor = 0

        if method == 'lsmr':
            result_tuple = lsmr(orbit_linear_operator, b, damp=damp, atol=atol, btol=btol,
                                conlim=conlim, maxiter=maxiter, show=show)
        elif method == 'lsqr':
            # Depends heavily on scaling of the problem.
            result_tuple = lsqr(orbit_linear_operator, b, damp=damp, atol=atol, btol=btol, conlim=conlim,
                                iter_lim=maxiter, show=show, calc_var=calc_var)
        else:
            raise ValueError('Unknown solver %s' % method)

        x = result_tuple[0]
        istop = result_tuple[1]
        n_iter += result_tuple[2]

        dorbit = orbit_.from_numpy_array(x, **kwargs)
        next_orbit = orbit_.increment(dorbit)
        next_residual = next_orbit.residual()
        while next_residual > residual:
            damp_factor += 1
            next_orbit = orbit_.increment(dorbit, stepsize=2 ** -damp_factor)
            next_residual = next_orbit.residual()
            if damp_factor > max_damp_factor:
                return orbit_, n_iter, 0
        else:
            orbit_ = next_orbit
            residual = next_residual
            if kwargs.get('verbose', False):
                if np.mod(orbit_n_iter, (orbit_maxiter // 10)) == 0:
                    print('Residual={} after {} {} iterations'.format(orbit_.residual(), orbit_n_iter, method))

    if orbit_.residual() <= orbit_tol:
        orbit_, exit_code = orbit_.verify_integrity()
    elif n_iter == orbit_maxiter:
        exit_code = 2
    else:
        exit_code = 0

    return orbit_, n_iter, exit_code


def _scipy_optimize_minimize_wrapper(orbit_, method=None, bounds=None,
                                     tol=None, callback=None, options=None, **kwargs):

    # The performance of the different methods depends on preconditioning differently/
    if method in ['newton-cg', 'tnc']:
        # This is a work-around to have different defaults for the different methods.
        preconditioning = kwargs.get('preconditioning', False)
    elif method =='l-bfgs-b':
        # This is a work-around to have different defaults for the different methods.
        preconditioning = kwargs.get('preconditioning', True)
    elif method =='cg':
        # This is a work-around to have different defaults for the different methods.
        preconditioning = kwargs.get('preconditioning', True)
        if options is None:
            options = {'gtol': 1e-3}
        elif isinstance(options, dict):
            options['gtol'] = options.get('gtol', 1e-3)

    def _cost_function_scipy_minimize(x):
        '''
        :param x0: (n,) numpy array
        :param args: time discretization, space discretization, subClass from orbit.py
        :return: value of cost functions (0.5 * L_2 norm of spatiotemporal mapping squared)
        '''

        '''
        Note that passing Class as a function avoids dangerous statements using eval()
        '''
        x_orbit = orbit_.from_numpy_array(x, **kwargs)
        return x_orbit.residual()

    def _cost_function_jac_scipy_minimize(x):
        """ The jacobian of the cost function (scalar) can be expressed as a vector product

        Parameters
        ----------
        x
        args

        Returns
        -------

        Notes
        -----
        The gradient of 1/2 F^2 = J^T F, rmatvec is a function which does this matrix vector product
        Will always use preconditioned version by default, not sure if wise.
        """

        x_orbit = orbit_.from_numpy_array(x)
        if preconditioning:
            return (x_orbit.rmatvec(x_orbit.spatiotemporal_mapping(), **kwargs)
                    ).precondition(x_orbit.parameters).state_vector().ravel()
        else:
            return x_orbit.rmatvec(x_orbit.spatiotemporal_mapping(), **kwargs).state_vector().ravel()

    orbit_n_iter = 0
    success = True
    while ((orbit_.residual() > kwargs.get('orbit_tol', orbit_.M * orbit_.N * 10**-6))
           and (orbit_n_iter < kwargs.get('orbit_maxiter', 20))
           and success):
        orbit_n_iter += 1
        result = minimize(_cost_function_scipy_minimize, orbit_.state_vector(),
                          method=method, jac=_cost_function_jac_scipy_minimize, bounds=bounds, tol=tol,
                          callback=callback, options=options)
        orbit_ = orbit_.from_numpy_array(result.x)
        success = result.success
        if kwargs.get('verbose', False):
            if np.mod(orbit_n_iter, (kwargs.get('orbit_maxiter', 20) // 10)) == 0:
                print('Residual={} after {} {} iterations'.format(orbit_.residual(), orbit_n_iter, method))

    return orbit_, orbit_n_iter, success


def _scipy_optimize_root_wrapper(orbit_, method=None, tol=None, callback=None, options=None, **kwargs):
    """ Wrapper for scipy.optimize.root methods

    Parameters
    ----------
    orbit_
    method
    tol
    callback
    options
    kwargs

    Returns
    -------

    Notes
    -----
    Does not allow for preconditioning currently. Only supports the following methods: 'lm', 'lgmres', 'gmres', 'minres'

    """
    # define the functions using orbit instance within scope instead of passing orbit
    # instance as arg to scipy functions.

    def _cost_function_scipy_root(x):
        '''
        :param x0: (n,) numpy array
        :param args: time discretization, space discretization, subClass from orbit.py
        :return: value of cost functions (0.5 * L_2 norm of spatiotemporal mapping squared)
        '''

        '''
        Note that passing Class as a function avoids dangerous statements using eval()
        '''
        x_orbit = orbit_.from_numpy_array(x, **kwargs)
        n_params = x_orbit.state_vector().size - x_orbit.state.size
        # Root requires input shape = output shape.
        return np.concatenate((x_orbit.spatiotemporal_mapping(**kwargs).state.ravel(),
                               np.zeros(n_params)), axis=0)

    def _cost_function_jac_scipy_root(x):
        """ The jacobian of the cost function (scalar) can be expressed as a vector product

        Parameters
        ----------
        x
        args

        Returns
        -------

        Notes
        -----
        The gradient of 1/2 F^2 = J^T F, rmatvec is a function which does this matrix vector product
        Will always use preconditioned version by default, not sure if wise.
        """

        x_orbit = orbit_.from_numpy_array(x, **kwargs)
        n_params = x_orbit.state_vector().size - x_orbit.state.size
        return np.concatenate((x_orbit.jacobian(**kwargs),
                               np.zeros([n_params, x_orbit.state_vector().size])), axis=0)

    # If not providing jacobian numerical approximation is used which can be beneficial.
    if method == 'lm' and kwargs.get('jacobian_on', True):
        jac_ = _cost_function_jac_scipy_root
    elif method in ['lgmres', 'gmres', 'minres']:
        jac_ = None
        # These methods are actually inner loop methods of 'krylov' method; need to be passed into options dict.
        #
        if options is None:
            options = {'jac_options': {'method': method}}
            method = 'krylov'
        elif isinstance(options, dict):
            options['method'] = method
            options['jac_options'] = {'method': method}
    else:
        jac_ = None

    orbit_n_iter = 0
    success = True
    while ((orbit_.residual() > kwargs.get('orbit_tol', orbit_.M * orbit_.N * 10**-6))
           and (orbit_n_iter < kwargs.get('orbit_maxiter', 20))
           and success):
        orbit_n_iter += 1
        result = root(_cost_function_scipy_root, orbit_.state_vector(),
                      method=method, jac=jac_, tol=tol,
                      callback=callback, options=options)
        orbit_ = orbit_.from_numpy_array(result.x, **kwargs)
        success = result.success
        if kwargs.get('verbose', False):
            if np.mod(orbit_n_iter, (kwargs.get('orbit_maxiter', 20) // 10)) == 0:
                print('Residual={} after {} {} iterations'.format(orbit_.residual(), orbit_n_iter, method))
    return orbit_, orbit_n_iter, success

In [3]:
test = read_h5('OrbitKS_L37p297_T79p778.h5', directory='local')
orbit_ = rediscretize(test, new_N=64, new_M=32)
np.random.seed(0)
orbit_ = orbit_ + OrbitKS(state=0.1*np.random.randn(*orbit_.state.shape), state_type='field')
orbit_ = orbit_.convert(to='modes')
cdict = {1:'success', 0:'failure', 2:'success'}
orbit_list = []
res, tim = [], []
max_iter = 3

In [4]:
methods = (['cg', 'newton-cg', 'l-bfgs-b', 'tnc']
           +['lm', 'lgmres', 'gmres', 'minres']
           +['lsqr','lsmr']+['gradient', 'lstsq'])

In [5]:
def method_efficiency(orbit_instance, method, maxiter, **kwargs):
    preconditioning = kwargs.get('preconditioning', False)
    print(method)
    print('\n')
    t0 = time.time()
    if method in ['cg', 'newton-cg', 'l-bfgs-b', 'tnc']:
        result_orbit, nit, success = _scipy_optimize_minimize_wrapper(orbit_, method=m,
                                                                      options={'maxiter':maxiter},
                                                                      preconditioning=preconditioning)
    elif method in ['lm', 'lgmres', 'gmres', 'minres']:
        result_orbit, nit, success = _scipy_optimize_root_wrapper(orbit_, method=m,
                                                                      options={'maxiter':maxiter},
                                                                      preconditioning=preconditioning)
    elif method in ['lsqr','lsmr']:
        result_orbit, nit, success = _scipy_sparse_linalg_solver_wrapper(orbit_, method=m,
                                                                      outer_maxiter=10,
                                                                      preconditioning=preconditioning)
    elif method in ['adj', 'lstsq']:
        result = converge(orbit_, method=m, maxiter=10, preconditioning=preconditioning)
        result_orbit = result.orbit
    else: 
        return 0, 0, 0, 0, 0
    
    t1 = time.time()
    print('\n')
    print('time', t1-t0, 'residual', result_orbit.residual(), '\n')
    print('########################################################')
    res = result_orbit.residual()
    tim = t1-t0
    # define as percentage reduction per second. 
    efficiency = (1 - (res/orbit_instance.residual())) / tim
    efficiency_log10 = np.log10(efficiency+1)
    return res, tim, efficiency, efficiency_log10, preconditioning

In [6]:
maxiter = 10
res_list, time_list, efficiency_list, efficiency_log10_list, preconditioning_list = [], [], [], [], []
for m in methods:
    res, tim, efficiency, efficiency_log10, preconditioning = method_efficiency(orbit_, m, maxiter, preconditioning=False)
    res_list.append(res)
    time_list.append(tim)
    efficiency_list.append(efficiency)
    efficiency_log10_list.append(efficiency_log10)
    preconditioning_list.append(preconditioning)

cg




time 0.044959306716918945 residual 16.86222568416023 

########################################################
newton-cg




time 0.5890178680419922 residual 0.5335763269976189 

########################################################
l-bfgs-b




time 0.0359654426574707 residual 12.78361847222743 

########################################################
tnc




time 0.023977041244506836 residual 44.570001670279005 

########################################################
lm




time 88.72746109962463 residual 0.10549197157506089 

########################################################
lgmres




ValueError: Unknown solver lgmres

In [ ]:
maxiter = 10
for m in methods:
    res, tim, efficiency, efficiency_log10, preconditioning = method_efficiency(orbit_, m, maxiter, preconditioning=True)
    res_list.append(res)
    time_list.append(tim)
    efficiency_list.append(efficiency)
    efficiency_log10_list.append(efficiency_log10)
    preconditioning_list.append(preconditioning)

In [ ]:
method_scoring_df = pd.DataFrame(np.concatenate((res_list, time_list, efficiency_list, efficiency_log10_list,
                                                 preconditioning_list)).reshape(5,-1).transpose(),
                                 index=methods+methods,
                                 columns=['residual', 'time', 'efficiency', 'log10_efficiency','preconditioning'])

In [ ]:
method_scoring_df.sort_values(by='log10_efficiency', ascending=False)

In [ ]:
method_scoring_df.loc[:, 'preconditioning'] = method_scoring_df.preconditioning.astype(bool).values
method_scoring_df.sort_values(by='log10_efficiency', ascending=False).to_csv('numerical_efficiency.csv')
method_scoring_df.sort_index().to_csv('numerical_efficiency_index_sort.csv')

In [ ]:
_ = method_scoring_df.groupby([method_scoring_df.index, method_scoring_df.preconditioning.astype(bool)]).sum().unstack().log10_efficiency.plot.bar(figsize=(10,10), fontsize=12)
loc, lab = plt.xticks()
_ = plt.xticks(loc, lab, ha='right', rotation=45)
plt.ylabel('Log10(Percentage residual decrease / time)')
plt.savefig('numerical_method_decrease_ratio_efficiency.jpg')